In [1]:
%%bash
dmmakereg region="region(ds9.reg)" out=test.fits ker=fits wcs=ngc3628_2039_broad_thresh.img cl+
punlearn roi
roi infile=test.fits outsrcfile=roi_out_%d.fits radiusmode=mul bkgradius=3 group=exclude targetbkg=target mo=h cl+

In [2]:
%%bash
dmlist roi_out\* counts

3       
3       
3       


In [3]:
from region import CXCRegion


In [4]:
aa = CXCRegion("roi_out_1.fits")
bb = CXCRegion("roi_out_2.fits")
cc = CXCRegion("roi_out_3.fits")

In [5]:
aa[0]

ellipse(4022.93919125,4005.69778265,7.25609756098,6.97764227642,359.99657)

**bug** in `__contains__` method, need to negate

In [6]:
aa[0] in bb  or ~aa[0] in bb

True

In [7]:
from pycrates import read_file

infile="ngc3628_2039_broad_thresh.img"

In [8]:
img = read_file(infile)

In [9]:
import stk
rois = stk.build("roi_out_?.fits")
rois_reg = [CXCRegion(r) for r in rois]

In [10]:
from ciao_contrib.runtool import make_tool
dmstat = make_tool("dmstat")

stats = []
for rr in rois_reg:
    dmstat.infile="{}[sky={}]".format(infile, rr[0])
    dmstat(centroid=False)
    stats.append(float(dmstat.out_sum))

In [11]:
stats

[120.0, 46.0, 18.0]

In [12]:
rois

['roi_out_1.fits', 'roi_out_2.fits', 'roi_out_3.fits']

In [13]:
rois_reg

[ellipse(4022.93919125,4005.69778265,7.25609756098,6.97764227642,359.99657)*!ellipse(4033.54533473,4001.51037323,7.81504065041,6.69715447154,359.99657)*!ellipse(4035.777398,4017.41883113,17.0243902439,11.7215447154,357.21175),
 ellipse(4033.54533473,4001.51037323,7.81504065041,6.69715447154,359.99657)*!ellipse(4022.93919125,4005.69778265,7.25609756098,6.97764227642,359.99657)*!ellipse(4035.777398,4017.41883113,17.0243902439,11.7215447154,357.21175),
 ellipse(4035.777398,4017.41883113,17.0243902439,11.7215447154,357.21175)*!ellipse(4022.93919125,4005.69778265,7.25609756098,6.97764227642,359.99657)*!ellipse(4033.54533473,4001.51037323,7.81504065041,6.69715447154,359.99657)]

In [14]:
roi0 = [rr[0] for rr in rois_reg]

In [15]:
roi0

[ellipse(4022.93919125,4005.69778265,7.25609756098,6.97764227642,359.99657),
 ellipse(4033.54533473,4001.51037323,7.81504065041,6.69715447154,359.99657),
 ellipse(4035.777398,4017.41883113,17.0243902439,11.7215447154,357.21175)]

In [16]:
from region import CXCRegion
class ROI():
    
    def __init__(self,filename):
        self.filename = filename
        self.region = CXCRegion(filename)
        self.r0 = self.region[0]
        self.metric = None
        self.del_other = []
        

    def compute_metric(self,propfile):
        from ciao_contrib.runtool import make_tool
        dmstat = make_tool("dmstat")
        dmstat.infile="{}[sky={}]".format(propfile, self.r0)
        dmstat(centroid=False)
        self.metric = float(dmstat.out_sum)
    
    
    def remove_from(self,other):                    
        # skip self
        if other.r0 == self.r0:
            return

        # skip if other is bigger
        if other.metric > self.metric:
            return
        
        idx = self.region.index(~other.r0)
        self.del_other.extend(idx)

        
    def reload_with_filter(self):
        if len(self.del_other)>0:
            ff = "{}[exclude #row={}]"
            # +1: array index -> dm row number
            rr = ",".join([str(x+1) for x in self.del_other]) 
            ff = ff.format(self.filename, rr)
            self.new_region = CXCRegion(ff)
        else:      
            self.new_region = CXCRegion(self.filename)
        
        
        
import stk
rois = stk.build("FalseStart/roi/out_0*")
rois_reg = [ROI(r) for r in rois]

infile="ngc3628_2039_broad_thresh.img"
for r in rois_reg:
    r.compute_metric(infile)

In [17]:

for r in rois_reg:
    for k in rois_reg:
        k.remove_from(r)

In [18]:

for r in rois_reg:
    r.reload_with_filter()

In [19]:
rois_reg[0].region

ellipse(4286.05015674,3550.07836991,37.7600364685,16.2104797363,37.342250824)*!ellipse(4300.7962963,3525.59259259,24.1764907837,15.7014369965,62.3044509888)

In [20]:
rois_reg[0].new_region


ellipse(4286.05015674,3550.07836991,37.7600364685,16.2104797363,37.342250824)